# Influence of weighting on the local prediction error

In [ ]:
from os.path import join
from os import makedirs
import torch as pt
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from flowtorch.analysis import DMD
from utils import add_oat_patch

plt.rcParams['figure.dpi'] = 160
plt.rc('text', usetex=True)

data = "/home/andre/Development/naca0012_shock_buffet/run/oat15"
path = "./output/oat15_analysis/local_error"

makedirs(path, exist_ok=True)

In [ ]:
times = pt.load(join(data, "oat15_tandem_times.pt"))[::20]
vertices = pt.load(join(data, "vertices_and_masks.pt"))
area = vertices["area_small"]
x = vertices["x_small"]
z = vertices["z_small"]
del vertices
start_at, end_at = 101, 501 # encloses 2 cycles

In [ ]:
vel_x = pt.load(join(data, "vel_x_small_every10.pt"))[:, start_at:end_at:2]
vel_z = pt.load(join(data, "vel_z_small_every10.pt"))[:, start_at:end_at:2]
dm = pt.cat((vel_x, vel_z), dim=0)
del vel_x, vel_z

In [ ]:
dt = times[1] - times[0]
rank = 65
dmd = DMD(dm, dt, rank=rank, optimal=True)
weights = area.sqrt().unsqueeze(-1).repeat((2, 1))
dmdw = DMD(dm*weights, dt, rank=rank, optimal=True)
print(dmd, dmdw)

In [ ]:
YH = (dmd.modes @ pt.diag(dmd.eigvals)) @ (pt.linalg.pinv(dmd.modes) @ dmd._dm[:, :-1].type(dmd.modes.dtype))
p_err = (YH.real - dm[:, 1:]).abs().sum(dim=1)
YH = (dmdw.modes @ pt.diag(dmdw.eigvals)) @ (pt.linalg.pinv(dmdw.modes) @ dmdw._dm[:, :-1].type(dmdw.modes.dtype))
p_err_w = (YH.real / weights - dm[:, 1:]).abs().sum(dim=1)

In [ ]:
p_err.min(), p_err.max(), p_err_w.min(), p_err_w.max()

In [ ]:
def fmt(x, pos):
    a, b = '{:.0e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

CHORD = 0.15
U_INF = 238.59
n_points = area.shape[0]
fig, axarr = plt.subplots(3, 1, figsize=(6, 4), sharex=True, sharey=True)
vmin, vmax = -250, 250
levels = pt.linspace(vmin, vmax, 100)
cont_0 = axarr[0].tricontourf(x/CHORD, z/CHORD, p_err[:n_points]-p_err_w[:n_points], cmap="seismic", levels=levels, vmin=vmin, vmax=vmax, extend="both")
cbar_0 = plt.colorbar(cont_0, ax=axarr[0], label=r"$E_{\mathrm{cum}} - E_{\mathrm{cum}}^w$", extend="both")
cbar_0.ax.get_yaxis().set_ticks([vmin, 0, vmax])
vmin, vmax = 5e-6, 1e-3
levels = pt.linspace(vmin, vmax, 100)
cont_1 = axarr[1].tricontourf(x/CHORD, z/CHORD, area.sqrt(), levels=levels, vmin=vmin, vmax=vmax, extend="max")
cbar_1 = plt.colorbar(cont_1, ax=axarr[1], label=r"$\sqrt{A}$", extend="max", format=ticker.FuncFormatter(fmt))
cbar_1.ax.get_yaxis().set_ticks([5e-6, 5e-4, 1e-3])
dm_std = dm.std(dim=1)[:n_points] / U_INF
vmin, vmax = 0.0, 0.9*dm_std.max()
levels = pt.linspace(vmin, vmax, 100)
cont_2 = axarr[2].tricontourf(x/CHORD, z/CHORD, dm_std, levels=levels, vmin=vmin, vmax=vmax, extend="max")
cbar_2 = plt.colorbar(cont_2, ax=axarr[2], label=r"$\sigma_{u_x}/U_\infty$", extend="max", format=r"$%1.2f$")
cbar_2.ax.get_yaxis().set_ticks([vmin, 0.5*(vmin+vmax), vmax])
for ax in axarr:
    ax.set_aspect("equal")
    add_oat_patch(ax)
    ax.set_ylim(-0.2, 0.5)
    ax.set_xlim(-0.1, 2.5)
fig.add_subplot(111, frame_on=False)
plt.tick_params(labelcolor="none", bottom=False, left=False)
plt.ylabel(r"$\tilde{z}$", labelpad=-10)
axarr[-1].set_xlabel(r"$\tilde{x}$")
axarr[0].text(2.3, 0.4, "(a)")
axarr[1].text(2.3, 0.4, "(b)")
axarr[2].text(2.3, 0.4, "(c)", c="w")
plt.savefig(join(path, "projection_error_comparison.png"), bbox_inches="tight")
plt.show()